In [1]:
import time
import pickle
import numpy as np
from src.environment import StockTradingEnvironment
from src.utils import save_pickle, load_pickle, plot_grid
from src.learner import q_learning_learning_loop
import yfinance as yf
import random
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import LearningRateScheduler
import logging
import multiprocessing as mp

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    logging.info("Using GPU for training")
else:
    logging.info("Using CPU for training")

2024-06-23 14:49:49,944 - INFO - Using GPU for training


In [ ]:
# yfinance download data
def fetch_stock_data(symbol, start_date, end_date, output_file):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['Close'] = stock_data['Adj Close']
    
    stock_data = stock_data.drop(columns=['Adj Close'])

    stock_data.to_csv(output_file)
        
    return stock_data

stock_data = fetch_stock_data('AAPL', '2019-06-20', '2024-06-21', 'AAPL_data.csv')

#### Q-Learning

In [2]:
class SumTree:
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        self.write = 0

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2
        self.tree[parent] += change
        if parent != 0:
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1
        if left >= len(self.tree):
            return idx
        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])

    def total(self):
        return self.tree[0]

    def add(self, p, data):
        idx = self.write + self.capacity - 1
        self.data[self.write] = data
        self.update(idx, p)
        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

    def update(self, idx, p):
        change = p - self.tree[idx]
        self.tree[idx] = p
        self._propagate(idx, change)

    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1
        return idx, self.tree[idx], self.data[dataIdx]

class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6):
        self.tree = SumTree(capacity)
        self.alpha = alpha
        self.capacity = capacity
        self.epsilon = 0.01

    def add(self, error, sample):
        p = (error + self.epsilon) ** self.alpha
        self.tree.add(p, sample)

    def sample(self, batch_size, beta=0.4):
        batch = []
        idxs = []
        segment = self.tree.total() / batch_size
        priorities = []
        for i in range(batch_size):
            a = segment * i
            b = segment * (i + 1)
            s = random.uniform(a, b)
            idx, p, data = self.tree.get(s)
            batch.append(data)
            idxs.append(idx)
            priorities.append(p)
        sampling_probabilities = priorities / self.tree.total()
        is_weights = np.power(self.tree.capacity * sampling_probabilities, -beta)
        is_weights /= is_weights.max()
        return batch, idxs, is_weights

    def update(self, idx, error):
        p = (error + self.epsilon) ** self.alpha
        self.tree.update(idx, p)

In [3]:
class DoubleQLearningAgent:
    def __init__(self, env, learning_rate, discount_factor):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.q_table1 = {}
        self.q_table2 = {}

    def step(self, state, epsilon):
        state_index = self._get_state_index(state)
        if np.random.rand() < epsilon:
            return np.random.choice(self.env.action_space.n)
        else:
            q_values = self.q_table1.get(state_index, np.zeros(self.env.action_space.n)) + self.q_table2.get(state_index, np.zeros(self.env.action_space.n))
            return np.argmax(q_values)

    def update_qvalue(self, state, action, reward, next_state):
        state_index = self._get_state_index(state)
        next_state_index = self._get_state_index(next_state)
        if np.random.rand() < 0.5:
            best_next_action = np.argmax(self.q_table1.get(next_state_index, np.zeros(self.env.action_space.n)))
            self.q_table1[state_index] = self.q_table1.get(state_index, np.zeros(self.env.action_space.n))
            self.q_table1[state_index][action] += self.learning_rate * (
                reward + self.discount_factor * self.q_table2.get(next_state_index, np.zeros(self.env.action_space.n))[best_next_action] - self.q_table1[state_index][action]
            )
        else:
            best_next_action = np.argmax(self.q_table2.get(next_state_index, np.zeros(self.env.action_space.n)))
            self.q_table2[state_index] = self.q_table2.get(state_index, np.zeros(self.env.action_space.n))
            self.q_table2[state_index][action] += self.learning_rate * (
                reward + self.discount_factor * self.q_table1.get(next_state_index, np.zeros(self.env.action_space.n))[best_next_action] - self.q_table2[state_index][action]
            )

    def _get_state_index(self, state):
        return tuple(state.flatten())

class DQNAgent:
    def __init__(self, env, learning_rate, discount_factor, buffer_size=10000, batch_size=32, alpha=0.6, beta=0.4):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.memory = PrioritizedReplayBuffer(buffer_size, alpha)
        self.beta = beta
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

    def build_model(self):
        model = models.Sequential()
        input_dim = np.prod(self.env.observation_space.shape)
        model.add(layers.Dense(24, input_dim=input_dim, activation='relu'))
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(self.env.action_space.n, activation='linear'))
        model.compile(optimizer=optimizers.Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        state = state.flatten().reshape(1, -1)
        next_state = next_state.flatten().reshape(1, -1)
        target = reward + self.discount_factor * np.amax(self.target_model.predict(next_state)[0]) if not done else reward
        td_error = abs(target - np.amax(self.model.predict(state)[0]))
        self.memory.add(td_error, (state, action, reward, next_state, done))

    def act(self, state, epsilon):
        if np.random.rand() <= epsilon:
            return np.random.choice(self.env.action_space.n)
        state = state.flatten().reshape(1, -1)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self):
        if len(self.memory.tree.data) < self.batch_size:
            return
        minibatch, idxs, is_weights = self.memory.sample(self.batch_size, self.beta)
        states, actions, rewards, next_states, dones = zip(*minibatch)
        states = np.array(states).reshape(self.batch_size, -1)
        next_states = np.array(next_states).reshape(self.batch_size, -1)
        targets = self.model.predict(states)
        targets_next = self.target_model.predict(next_states)

        for i in range(self.batch_size):
            target = rewards[i] + self.discount_factor * np.amax(targets_next[i]) if not dones[i] else rewards[i]
            td_error = abs(target - targets[i][actions[i]])
            self.memory.update(idxs[i], td_error)
            targets[i][actions[i]] = target

        self.model.fit(states, targets, epochs=1, verbose=0)
        self.update_target_model()

In [4]:
def collect_experience(env, agent, epsilon):
    obs, _ = env.reset()
    terminated, truncated = False, False
    experiences = []

    while not terminated and not truncated:
        current_action = agent.step(obs, epsilon)
        next_obs, reward, terminated, truncated, _ = env.step(current_action)
        experiences.append((obs, current_action, reward, next_obs, terminated))
        obs = next_obs
    
    return experiences

def adaptive_learning_rate(epoch, lr):
    if epoch % 100 == 0 and epoch:
        return lr * 0.1
    return lr

def decay_epsilon(episode, epsilon, min_epsilon, decay_rate):
    return max(min_epsilon, epsilon * decay_rate)

def q_learning_learning_loop(env, agent, learning_rate: float, discount_factor: float, episodes: int,
                             min_epsilon_allowed: float, initial_epsilon_value: float,
                             buffer_size: int = 10000, batch_size: int = 32, decay_method="exponential") -> tuple:
    epsilon = initial_epsilon_value
    epsilon_decay_factor = np.power(min_epsilon_allowed / epsilon, 1 / episodes)

    reward_across_episodes = []
    epsilons_across_episodes = []

    replay_buffer = deque(maxlen=buffer_size)

    for episode in range(episodes):
        experiences = collect_experience(env, agent, epsilon)
        replay_buffer.extend(experiences)

        if len(replay_buffer) >= batch_size:
            batch = np.array(random.sample(replay_buffer, batch_size), dtype=object)
            for b_state, b_action, b_reward, b_next_state, b_done in batch:
                b_state = tuple(b_state.flatten())  # Convert to tuple
                b_next_state = tuple(b_next_state.flatten())  # Convert to tuple
                if b_done:
                    target = b_reward
                else:
                    target = b_reward + discount_factor * np.max(agent.q_table1.get(b_next_state, np.zeros(env.action_space.n)) + agent.q_table2.get(b_next_state, np.zeros(env.action_space.n)))

                agent.q_table1[b_state] = agent.q_table1.get(b_state, np.zeros(env.action_space.n))
                agent.q_table1[b_state][b_action] += learning_rate * (target - agent.q_table1[b_state][b_action])

        if decay_method == "exponential":
            epsilon = decay_epsilon(episode, epsilon, min_epsilon_allowed, epsilon_decay_factor)
        else:
            epsilon = max(min_epsilon_allowed, epsilon - (initial_epsilon_value - min_epsilon_allowed) / episodes)
        
        reward_across_episodes.append(sum([exp[2] for exp in experiences]))
        epsilons_across_episodes.append(epsilon)

        if (episode + 1) % 100 == 0:
            logging.info(f"Episode {episode + 1}/{episodes} - Reward: {reward_across_episodes[-1]} - Epsilon: {epsilon}")

    logging.info("Trained Q-Table: %s", agent.q_table1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(reward_across_episodes, label='Rewards')
    plt.xlabel('Episodes')
    plt.ylabel('Cumulative Reward')
    plt.title('Rewards over Episodes')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epsilons_across_episodes, label='Epsilon')
    plt.xlabel('Episodes')
    plt.ylabel('Epsilon')
    plt.title('Epsilon Decay over Episodes')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return agent, reward_across_episodes, epsilons_across_episodes

In [5]:
def parallel_collect_experience(env, agent, epsilon, num_workers=4):
    with mp.Pool(num_workers) as pool:
        results = pool.starmap(collect_experience, [(env, agent, epsilon) for _ in range(num_workers)])
    experiences = [exp for result in results for exp in result]
    return experiences

In [6]:
# NOTE: You can adjust the parameter 'number_of_days_to_consider'

env = stock_trading_environment = StockTradingEnvironment('./AAPL_data.csv', number_of_days_to_consider=30)

In [7]:
# Example with a higher batch size of 256
agent = DoubleQLearningAgent(env, learning_rate=0.001, discount_factor=0.95)

agent, reward_across_episodes, epsilons_across_episodes = q_learning_learning_loop(
    env,
    agent,
    learning_rate=0.001,
    discount_factor=0.95,
    episodes=50000,
    min_epsilon_allowed=0.01,
    initial_epsilon_value=1,
    buffer_size=100000,
    batch_size=256,  # Increased batch size
    decay_method="exponential"
)

2024-06-23 14:51:05,462 - INFO - Episode 100/50000 - Reward: -1585.7576360484668 - Epsilon: 0.9908319448927725
2024-06-23 14:52:03,918 - INFO - Episode 200/50000 - Reward: -1554.3286083740397 - Epsilon: 0.9817479430199942
2024-06-23 14:53:04,011 - INFO - Episode 300/50000 - Reward: -1688.6333644746494 - Epsilon: 0.9727472237769801
2024-06-23 14:54:03,581 - INFO - Episode 400/50000 - Reward: -1752.4343927235489 - Epsilon: 0.9638290236239909
2024-06-23 14:55:02,515 - INFO - Episode 500/50000 - Reward: -1398.4896248442853 - Epsilon: 0.954992586021461
2024-06-23 14:56:02,025 - INFO - Episode 600/50000 - Reward: -1680.3684293335418 - Epsilon: 0.9462371613658228
2024-06-23 14:57:00,873 - INFO - Episode 700/50000 - Reward: -1573.8371964666544 - Epsilon: 0.937562006925914
2024-06-23 14:57:59,970 - INFO - Episode 800/50000 - Reward: -1556.4228127668825 - Epsilon: 0.9289663867799748
2024-06-23 14:58:59,182 - INFO - Episode 900/50000 - Reward: -1631.7386836587145 - Epsilon: 0.9204495717532143
202

In [ ]:
def run_learned_policy(env, agent):
    obs, _ = env.reset()
    terminated, truncated = False, False
    total_reward = 0
    steps = 0

    while not terminated and not truncated:
        obs_tuple = tuple(obs.flatten())
        action = np.argmax(agent.q_table1.get(obs_tuple, np.zeros(env.action_space.n)) + agent.q_table2.get(obs_tuple, np.zeros(env.action_space.n)))
        obs, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        steps += 1

    logging.info("Total Reward: %d, Steps: %d", total_reward, steps)
    return total_reward

total_reward = run_learned_policy(env, agent)

In [ ]:
def plot_grid(env, agent, reward_across_episodes: list, epsilons_across_episodes: list) -> None:
    env.train = False
    total_reward_learned_policy = [run_learned_policy(env, agent) for _ in range(30)]

    plt.figure(figsize=(15, 10))

    # Main plot
    plt.subplot(2, 2, 1)
    plt.plot(reward_across_episodes, 'ro')
    plt.xlabel('Episode')
    plt.ylabel('Reward Value')
    plt.title('Rewards Per Episode (Training)')
    plt.grid()

    plt.subplot(2, 2, 2)
    plt.plot(total_reward_learned_policy, 'ro')
    plt.xlabel('Episode')
    plt.ylabel('Reward Value')
    plt.title('Rewards Per Episode (Learned Policy Evaluation)')
    plt.grid()

    # Extra plots
    plt.subplot(2, 2, 3)
    plt.plot(reward_across_episodes)
    plt.xlabel('Episode')
    plt.ylabel('Cumulative Reward Per Episode (Training)')
    plt.title('Cumulative Reward vs Episode')
    plt.grid()

    plt.subplot(2, 2, 4)
    plt.plot(epsilons_across_episodes)
    plt.xlabel('Episode')
    plt.ylabel('Epsilon Values')
    plt.title('Epsilon Decay')
    plt.grid()

    plt.tight_layout()
    plt.show()

plot_grid(stock_trading_environment, agent, reward_across_episodes, epsilons_across_episodes)

In [ ]:
stock_trading_environment = StockTradingEnvironment('./AAPL_data.csv', number_of_days_to_consider=30)
stock_trading_environment.train = False
obs, _ = stock_trading_environment.reset()
terminated, truncated = False, False
while not terminated:
    # Convert observation to a tuple
    obs_tuple = tuple(obs.flatten())
    action = np.argmax(agent.q_table1.get(obs_tuple, np.zeros(env.action_space.n)) + agent.q_table2.get(obs_tuple, np.zeros(env.action_space.n)))
    obs, reward, terminated, truncated, info = stock_trading_environment.step(action)

stock_trading_environment.render()

In [ ]:
save_pickle(agent, 'aapl_q_learning_agent.pkl')

#### Forecast

In [ ]:
agent = load_pickle("aapl_q_learning_agent.pkl")

In [ ]:
stock_trading_environment = StockTradingEnvironment('./AAPL_data.csv', number_of_days_to_consider=30)
stock_trading_environment.train = False
obs, _ = stock_trading_environment.reset()
terminated, truncated = False, False
while not terminated:
    # Convert observation to a tuple
    obs_tuple = tuple(obs.flatten())
    action = np.argmax(agent.q_table1.get(obs_tuple, np.zeros(env.action_space.n)) + agent.q_table2.get(obs_tuple, np.zeros(env.action_space.n)))
    obs, reward, terminated, truncated, info = stock_trading_environment.step(action)

stock_trading_environment.render()